# Sensitivity Analysis Made Easy with the EMA Workbench

Google Colab version

This is the main notebook of the workshop on *sensitivity analysis* (SA) at the Social Simulation Festival 2021. Here we will demonstrate how to do Variance-based SA also know as [Sobol SA](https://en.wikipedia.org/wiki/Variance-based_sensitivity_analysis) on a relatively simple model [virus on network](https://ccl.northwestern.edu/netlogo/models/VirusonaNetwork). The idea is that you reuse (read copy-paste) this code your own model. Therefore, we tried to keep simple and avoid extensive side steps from.

This notebook is tuned to be run on [Google Colab](https://colab.research.google.com/) and has a couple of extra lines of code. If you want to use it on your local machine please use `sa_demo_local_machine.ipynb`.

The core packages used in this notebook are [Mesa](https://mesa.readthedocs.io/en/stable/) to define an ABM model in Python, [EMA Workbench](https://emaworkbench.readthedocs.io/en/latest/) to design and run experiments, [SALib](https://salib.readthedocs.io/en/latest/) to conduct SA (within EMA Workbench). Also, we used one pretty plotting function of [pyNetLogo](https://pynetlogo.readthedocs.io/en/latest/).

The notebook follows a simplified SA workflow and has 5 sections-steps:

<img src="img/workflow.png"/>

## 0. Installations and imports

In [1]:
# Clone the repo to make its file available for Google Colab
!git clone https://github.com/BROSE-Uninc/SSF2021.git

fatal: destination path 'SSF2021' already exists and is not an empty directory.


In [2]:
# Install necessary packages
!pip install ema_workbench mesa ipyparallel SALib &> /dev/null

^C


In [3]:
# Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import random

# Import EMA Workbench modules
from ema_workbench import ReplicatorModel, RealParameter, BooleanParameter, IntegerParameter, Constant, TimeSeriesOutcome, perform_experiments, save_results, ema_logging

# Initialize logger to keep track of experiments run
ema_logging.log_to_stderr(ema_logging.INFO)

# Import Mesa virus on network model
from SSF2021.wolf_sheep import model

/Users/kleinraphael/opt/anaconda3/lib/python3.8/site-packages/ema_workbench/em_framework/optimization.py:48: ImportWarning: platypus based optimization not available
  warnings.warn("platypus based optimization not available", ImportWarning)


## 1. Load the model

The very first step of SA with EMA Workbench is to define or "load" the model as a function. That is, EMA Workbench treats all models as functions (read *black box*). They are supposed to have **inputs** (parameters, constants, uncertainties and policy levers) and **outputs** (outcomes, KPIs). The model structure is not interesting for EMA Workbench. It may be something simple as `def func(x)` which just returns x + 1.

In [5]:
# setting up the wolf-sheep model simulation as a function
def model_wolf_sheep(height=20,
                     width=20,
                     initial_sheep=100,
                     initial_wolves=50,
                     sheep_reproduce=0.04,
                     wolf_reproduce=0.05,
                     wolf_gain_from_food=20,
                     grass=False,
                     grass_regrowth_time=30,
                     sheep_gain_from_food=4,
                     steps=200):
    
    from wolf_sheep import model
    
    # Initialising the model
    wolf_sheep = model.WolfSheep(height=20,
                                   width=20,
                                   initial_sheep=100,
                                   initial_wolves=50,
                                   sheep_reproduce=0.04,
                                   wolf_reproduce=0.05,
                                   wolf_gain_from_food=20,
                                   grass=False,
                                   grass_regrowth_time=30,
                                   sheep_gain_from_food=4)
                
    # Run the model steps times
    wolf_sheep.run_model(steps)
    
    # Get model outcomes
    outcomes = wolf_sheep.datacollector.get_model_vars_dataframe()
    
    # Return model outcomes
    # below to be changed!
    return {'TIME' : list(range(steps + 1)),
            "Wolves" : outcomes["Wolves"].tolist(),
            "Sheep" : outcomes["Sheep"].tolist()}
    

Now, let's parameterize and test out our Mesa model. What is supposed to happen? First, we shouldn't get any error 😅. Second, after we run `model_wolf_sheep` function it has to give us a set of model outcomes. Let's try.

In [6]:
# Parametrize the model
height=20
width=20
initial_sheep=100
initial_wolves=50
sheep_reproduce=0.04
wolf_reproduce=0.05
wolf_gain_from_food=20
grass=False
grass_regrowth_time=30
sheep_gain_from_food=4
steps=200

model_wolf_sheep(height=height, 
                 width=width, 
                 initial_sheep=initial_sheep, 
                 initial_wolves=initial_wolves, 
                 sheep_reproduce=sheep_reproduce, 
                 wolf_reproduce=wolf_reproduce, 
                 wolf_gain_from_food=wolf_gain_from_food,
                 grass=grass,
                 grass_regrowth_time=grass_regrowth_time,
                 sheep_gain_from_food=sheep_gain_from_food,
                 steps=steps)

{'TIME': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  1

## 2. Design experiments

Now it's time to design experiments. What does it mean? Well, we have to specify:

* **which model parameters** aka *inputs* are we going to sample, what are their **ranges**, and random **distributions**,
* what we will keep as **constants** and do not change over the model run,
* and finally which **outcomes** we want to observe.

It's an important step in SA workflow and we have to be careful. Because if parameter ranges are too narrow or they're sampled from e.g. a Normal distribution, there is a chance that you'll overlook import model behavior. This is why model parameters are named **uncertainties** in the EMA Workbench. We often do not know parameter vales and how to explore many plausible options.

Now let's talk about "tech" part. First we have to initialize an instance of EMA Workbench called `ReplicatorModel`. This is how we "connect" EMA Workbench to our Python model. We have to pass a name of our model to `ReplicatorModel`, and also pass the function that we defined previously.

In [7]:
# loading the model into the EMA workbench
model = ReplicatorModel(..., function=...)

SyntaxError: invalid syntax (<ipython-input-7-63d2f6f358b8>, line 2)

In [ ]:
# # defining ranges for the parameters to be varied
model.uncertainties = [RealParameter(..., ..., ...)]

In [ ]:
# defining the parameters that will remain constant
parameter_A = ...
model.constants = [Constant("parameter_A", parameter_A)]

In [ ]:
# defining the outputs of interests from the model
model.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('Wolves'),
                  TimeSeriesOutcome('Sheep')]

In [ ]:
# defining the number of replications
model.replications = 10

## 3. Run the model

In [ ]:
# running the EMA experiments with the aforementioned parameters and outputs
results = perform_experiments(model, 100)

In [ ]:
# opening the results
experiments, outcomes = results

In [ ]:
experiments.head()

In [ ]:
outcomes.keys()

### Epic save

In [ ]:
from ema_workbench.util.utilities import save_results, load_results

In [ ]:
# Save the results
save_results(results, 'results/results.tar.gz')

In [ ]:
# Load the results
results = load_results('results/results.tar.gz')

In [ ]:
experiments, outcomes = results

### A bit of preprocessing

In [ ]:
print(random.choice(list(outcomes)))
outcomes[random.choice(list(outcomes))].shape

In [ ]:
mean_outcomes = {key:np.mean(outcomes[key],axis=1) for key in outcomes.keys()}
mean_results = (experiments.copy(), mean_outcomes)

In [ ]:
# Now the shape of this array doesn't have 10 in it  
mean_outcomes[random.choice(list(outcomes))].shape

### Visuals!

In [ ]:
from ema_workbench.analysis.plotting import lines

In [ ]:
# plotting all of the results
plt.rcParams['figure.figsize'] = [10, 12]

figure = lines(experiments, outcomes_2D) #show lines, and end state density
plt.show()

## 4. Sensitivity analysis

In [ ]:
from SALib.analyze import sobol
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from src.plot import plot_sobol_indices
sns.set_style('white')

In [ ]:
# Specify the problem
problem = get_SALib_problem(model.uncertainties)

In [ ]:
# Select and normalize an outcome
normalized_outcomes = ...

In [ ]:
# Perform Sobol SA
Si = sobol.analyze(problem=problem, Y=normalized_resistant,
                   calc_second_order=True, print_to_console=False)

# Get scores by type 
Si_filter = {k:Si[k] for k in ['ST', 'ST_conf', 'S1', 'S1_conf']}

# Create a DataFrame out of them
Si_df = pd.DataFrame(Si_filter, index=problem['names'])

# Get indices and error bars
indices = Si_df[['S1','ST']]
err = Si_df[['S1_conf','ST_conf']]

In [ ]:
# Plot the results
fig, ax = plt.subplots(1)
indices.plot.bar(yerr=err.values.T,ax=ax)
fig.set_size_inches(8,6)
fig.subplots_adjust(bottom=0.3)

In [ ]:
sns.set_style('whitegrid')
fig = plot_sobol_indices(Si, problem, criterion='ST', threshold=0.005)
fig.set_size_inches(7,7)
plt.show()